In [1]:
import pandas as pd
import numpy as np

In [3]:
file_path = ".\\Base de dados\\archive\\accepted_2007_to_2018q4.csv\\accepted_2007_to_2018Q4.csv"
df = pd.read_csv(file_path)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_10692\982243736.py:2: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
colunas = ['loan_amnt', 'loan_status', 'home_ownership', 'purpose', 'addr_state', 'verification_status', \
                    'annual_inc', 'dti', 'fico_range_low', 'fico_range_high', 'open_acc', 'revol_bal', 'revol_util', 'emp_length']

df_filtrado = df[colunas].copy()

df_filtrado.dropna(subset=colunas, inplace=True)

nulos = df_filtrado[colunas].isnull().any(axis=1).sum()
print(nulos)


In [ ]:
df_filtrado['fico_range_avg'] = (df_filtrado['fico_range_low'] + df_filtrado['fico_range_high']) / 2

df_filtrado['loan_status_bin'] = df_filtrado['loan_status'].apply(
    lambda x: 1 if x in ['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)','Does not meet the credit policy. Status:Charged Off'] else 0
)

df_filtrado.to_csv('df_filtrado.csv', index=False)


In [3]:
df_filtrado = pd.read_csv("C:\\Users\\Gabriel\\Documents\\Faculdade\\TCC\\Cod\\df_filtrado.csv")

var_x = ['loan_amnt', 'home_ownership', 'purpose', 'addr_state', 'verification_status', \
                    'annual_inc', 'dti', 'open_acc', 'revol_bal', 'revol_util', 'emp_length', 'fico_range_avg']

x = df_filtrado.loc[:, var_x]
y = df_filtrado['loan_status_bin']

print(y.value_counts(normalize=True))

loan_status_bin
0    0.871437
1    0.128563
Name: proportion, dtype: float64


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, OrdinalEncoder

categoricas = ['home_ownership', 'purpose', 'addr_state', 'verification_status', 'emp_length']
numericas = ['loan_amnt', 'annual_inc', 'dti', 'fico_range_avg', 'open_acc', 'revol_bal', 'revol_util']

#Label Encoder = Ordinal Encoder
#Normalização = MinMaxScaler
#Padronização = Standart Scaler

preprocessador1 = ColumnTransformer(transformers=[
    ('num', MinMaxScaler(), numericas),
    ('cat', OrdinalEncoder(), categoricas)
], sparse_threshold=0)

preprocessador2 = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numericas),
    ('cat', OrdinalEncoder(), categoricas)
], sparse_threshold=0)

preprocessador3 = ColumnTransformer(transformers=[
    ('num', MinMaxScaler(), numericas),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categoricas)
], sparse_threshold=0)

preprocessador4 = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numericas),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categoricas)
], sparse_threshold=0)

preprocessadores = {
    'preprocessador 1': preprocessador1,
    'preprocessador 2': preprocessador2,
    'preprocessador 3': preprocessador3,
    'preprocessador 4': preprocessador4
}

In [ ]:
#---------------------------------Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

resultadosNB = []

for nome_prep, preprocessador in preprocessadores.items():
    pipelineNB = Pipeline([
        ("preprocessador", preprocessador),
        ("modelo", GaussianNB())
    ])

    resultado = cross_validate(
        pipelineNB, x, y, cv=5,
        scoring=['accuracy', 'recall_macro'],
        return_train_score=False
    )

    resultadosNB.append({
        'Preprocessador': nome_prep,
        'Accuracy': resultado['test_accuracy'].mean(),
        'Recall': resultado['test_recall_macro'].mean(),
        'Fit Time': resultado['fit_time'].mean(),
        'Score Time': resultado['score_time'].mean()
    })

df_resultadosNB = pd.DataFrame(resultadosNB)
df_resultadosNB = df_resultadosNB.sort_values(by='Preprocessador', ascending=True)

print("\n======= Resultados por pre-processador =======")
print(df_resultadosNB.to_string(index=False))

media_acc = df_resultadosNB['Accuracy'].mean()
media_recall = df_resultadosNB['Recall'].mean()
media_fit_time = df_resultadosNB['Fit Time'].mean()
media_score_time = df_resultadosNB['Score Time'].mean()

print("\n======= Média por pre-processador =======")
print(f"Média Accuracy: {media_acc:.4f}")
print(f"Média Recall: {media_recall:.4f}")
print(f"Média Fit Time:     {media_fit_time:.4f} segundos")
print(f"Média Score Time:   {media_score_time:.4f} segundos")

melhor = df_resultadosNB.loc[df_resultadosNB['Recall'].idxmax()]

print("\n" + "="*60)
print(" ======= Melhor resultado =======")
print("="*60)
print(f"{'Preprocessador:':25} {melhor['Preprocessador']}")
print(f"{'Accuracy:':25} {melhor['Accuracy']:.4f}")
print(f"{'Recall:':25} {melhor['Recall']:.4f}")
print(f"{'Fit Time:':25} {melhor['Fit Time']:.4f} segundos")
print(f"{'Score Time:':25} {melhor['Score Time']:.4f} segundos")
print("="*60)

df_resultadosNB.to_csv('df_resultadosNB.csv', index=False, sep=',')



======= Resultados por pre-processador =======
  Preprocessador  Accuracy   Recall  Fit Time  Score Time
preprocessador 1  0.866221 0.505498  3.241317    0.658211
preprocessador 2  0.859682 0.510967  2.884070    0.596061
preprocessador 3  0.520724 0.561078  5.558237    1.307618
preprocessador 4  0.524933 0.562239  5.183405    1.312365

======= Média por pre-processador =======
Média Accuracy: 0.6929
Média Recall: 0.5349
Média Fit Time:     4.2168 segundos
Média Score Time:   0.9686 segundos

 ======= Melhor resultado =======
Preprocessador:           preprocessador 4
Accuracy:                 0.5249
Recall:                   0.5622
Fit Time:                 5.1834 segundos
Score Time:               1.3124 segundos


In [ ]:
#---------------------------------Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate

resultadosDT = []

for nome_prep, preprocessador in preprocessadores.items():
    pipelineDT = Pipeline([
        ("preprocessador", preprocessador),
        ("modelo", DecisionTreeClassifier(class_weight='balanced'))
    ])

    resultado = cross_validate(
        pipelineDT, x, y, cv=5,
        scoring=['accuracy', 'recall_macro'],
        return_train_score=False
    )

    resultadosDT.append({
        'Preprocessador': nome_prep,
        'Accuracy': resultado['test_accuracy'].mean(),
        'Recall': resultado['test_recall_macro'].mean(),
        'Fit Time': resultado['fit_time'].mean(),
        'Score Time': resultado['score_time'].mean()
    })

df_resultadosDT = pd.DataFrame(resultadosDT)
df_resultadosDT = df_resultadosDT.sort_values(by='Preprocessador', ascending=True)

print("\n======= Resultados por pre-processador =======")
print(df_resultadosDT.to_string(index=False))

media_acc = df_resultadosDT['Accuracy'].mean()
media_recall = df_resultadosDT['Recall'].mean()
media_fit_time = df_resultadosDT['Fit Time'].mean()
media_score_time = df_resultadosDT['Score Time'].mean()

print("\n======= Media por pre-processador =======")
print(f"Média Accuracy: {media_acc:.4f}")
print(f"Média Recall: {media_recall:.4f}")
print(f"Média Fit Time:     {media_fit_time:.4f} segundos")
print(f"Média Score Time:   {media_score_time:.4f} segundos")


melhor = df_resultadosDT.loc[df_resultadosDT['Recall'].idxmax()]

print("\n" + "="*60)
print(" ======= Melhor resultado =======")
print("="*60)
print(f"{'Preprocessador:':25} {melhor['Preprocessador']}")
print(f"{'Accuracy:':25} {melhor['Accuracy']:.4f}")
print(f"{'Recall:':25} {melhor['Recall']:.4f}")
print(f"{'Fit Time:':25} {melhor['Fit Time']:.4f} segundos")
print(f"{'Score Time:':25} {melhor['Score Time']:.4f} segundos")
print("="*60)

df_resultadosDT.to_csv('df_resultadosDT.csv', index=False, sep=',')


In [ ]:
#---------------------------------Random Forest
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score
from sklearn.ensemble import RandomForestClassifier

modelo_parametrosRF = {
    'Random Forest': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'n_estimators': [10, 50 ,100],
            'class_weight': ['balanced'],
            'min_samples_leaf': [7],
            'min_samples_split': [2]
        }
    }
}

detalhes_RF = []

for nome_prep, preprocessador in preprocessadores.items():
    x_prep = x.copy()
    x_prep = preprocessador.fit_transform(x_prep)
    x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x_prep, y, test_size=0.15, random_state=0)

    for nome_modelo, objeto in modelo_parametrosRF.items():

        grid = GridSearchCV(objeto['modelo'], objeto['parametros'], cv=5, scoring='recall_macro')
        grid.fit(x_treinamento, y_treinamento)

        df_RF= pd.DataFrame(grid.cv_results_)
        y_pred = grid.predict(x_teste)

        df_RF['modelo'] = nome_modelo
        df_RF['preprocessador'] = nome_prep
        df_RF['n_estimators'] = df_RF['params'].apply(lambda x: x['n_estimators'])
        
        recall_scores = []
        for params in df_RF['params']:
            modelo = RandomForestClassifier(**params)
            modelo.fit(x_treinamento, y_treinamento)
            y_pred = modelo.predict(x_teste)
            recall = recall_score(y_teste, y_pred, average='macro')
            recall_scores.append(recall)

        df_RF['Recall'] = recall_scores

        detalhes_RF.append(df_RF[[
            'modelo', 'preprocessador', 'n_estimators',
            'mean_test_score', 'Recall', 'mean_fit_time', 'mean_score_time',
        ]])

        

df_detalhadoRF= pd.concat(detalhes_RF, ignore_index=True)
df_detalhadoRF = df_detalhadoRF.sort_values(by=['preprocessador', 'n_estimators'], ascending=[True, True])

for n_estimators, df_sub in df_detalhadoRF.groupby('n_estimators'):
    print(f"\n=== Numero de arvores: {n_estimators}")
    print(df_sub.to_string(index=False))

    for preproc, df_sub_prep in df_sub.groupby('preprocessador'):
        print(f"\n--- Resultados para Preprocessador: {preproc} ---")
        print(df_sub_prep.to_string(index=False))

    media_acc = df_sub['mean_test_score'].mean()
    media_recall = df_sub['Recall'].mean()
    print(f"Média Accuracy: {media_acc:.4f} | Média Recall: {media_recall:.4f}")

print("\n======= Media por pre-processador =======")
medias_por_preprocessador = df_detalhadoRF.groupby('preprocessador')[['mean_test_score', 'Recall']].mean().reset_index()

for _, linha in medias_por_preprocessador.iterrows():
    print(f"Preprocessador: {linha['preprocessador']} | Média Accuracy: {linha['mean_test_score']:.4f} | Média Recall: {linha['Recall']:.4f}")


media_recall_total = df_detalhadoRF['Recall'].mean()
media_acc_total = df_detalhadoRF['mean_test_score'].mean()
print(f"\n =======  Media total ======= \n Accuracy: {media_acc_total:.4f} | Recall: {media_recall_total:.4f}")

melhor_linhaRF = df_detalhadoRF.loc[df_detalhadoRF['Recall'].idxmax()]

print("\n======= Melhor Resultado Recall =======")
print(f"Preprocessador: {melhor_linhaRF['preprocessador']}")
print(f"Modelo: {melhor_linhaRF['modelo']}")
print(f"n_estimators: {melhor_linhaRF['n_estimators']}")
print(f"Accuracy (mean_test_score): {melhor_linhaRF['mean_test_score']:.4f}")
print(f"Recall-score: {melhor_linhaRF['Recall']:.4f}")
print(f"Tempo médio de treino: {melhor_linhaRF['mean_fit_time']:.4f} s")
print(f"Tempo médio de score: {melhor_linhaRF['mean_score_time']:.4f} s")


df_detalhadoRF.to_csv('df_resultadosRF.csv', index=False, sep=',')


In [6]:
#---------------------------------SVM
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score
from sklearn.svm import SVC

amostra = x.copy()
amostra['target'] = y
amostra = amostra.sample(50000, random_state=0)

x_sample = amostra.drop('target', axis=1)
y_sample = amostra['target']

modelo_parametrosSVM = {
    'SVM': {
        'modelo': SVC(),
        'parametros': {
            'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
            'C': [0.5, 1.0, 1.5],
            'class_weight': ['balanced'],
            'cache_size': [500]
        }
    }
}

detalhes_SVM = []

for nome_prep, preprocessador in preprocessadores.items():
    x_sample_prep = x_sample.copy()
    x_sample_prep = preprocessador.fit_transform(x_sample_prep)
    x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x_sample_prep, y_sample, test_size=0.15, random_state=0)

    for nome_modelo, objeto in modelo_parametrosSVM.items():
        
        grid = GridSearchCV(objeto['modelo'], objeto['parametros'], cv=5, scoring='recall_macro', n_jobs=-1)
        grid.fit(x_treinamento, y_treinamento)

        df_SVM = pd.DataFrame(grid.cv_results_)
        y_pred = grid.predict(x_teste)

        df_SVM['modelo'] = nome_modelo
        df_SVM['preprocessador'] = nome_prep
        df_SVM['kernel'] = df_SVM['params'].apply(lambda x: x['kernel'])
        df_SVM['C'] = df_SVM['params'].apply(lambda x: x['C'])
        
        recall_scores = []
        for params in df_SVM['params']:
            modelo = SVC(**params)
            modelo.fit(x_treinamento, y_treinamento)
            y_pred = modelo.predict(x_teste)
            recall = recall_score(y_teste, y_pred, average='weighted')
            recall_scores.append(recall)

        df_SVM['Recall'] = recall_scores

        detalhes_SVM.append(df_SVM[[
            'modelo', 'preprocessador', 'kernel', 'C',
            'mean_test_score', 'Recall', 'mean_fit_time', 'mean_score_time',
        ]])

        

df_detalhadoSVM = pd.concat(detalhes_SVM, ignore_index=True)
df_detalhadoSVM = df_detalhadoSVM.sort_values(by=['preprocessador', 'kernel', 'C'], ascending=[True, True, False])


for (preproc, kernel), df_sub in df_detalhadoSVM.groupby(['preprocessador', 'kernel']):
    print(f"\n=== Preprocessador: {preproc} | Kernel: {kernel} ===")
    print(df_sub.to_string(index=False))

    media_acc = df_sub['mean_test_score'].mean()
    media_recall = df_sub['Recall'].mean()
    print(f"\n>>> Medias para o Preprocessador: {preproc} | Kernel: {kernel} <<<")
    print(f"Média Accuracy: {media_acc:.4f} | Média Recall: {media_recall:.4f}")

media_recall_total = df_detalhadoSVM['Recall'].mean()
media_acc_total = df_detalhadoSVM['mean_test_score'].mean()


print("\n======= Media por pre-processador =======")
medias_por_preprocessador = df_detalhadoSVM.groupby('preprocessador')[['mean_test_score', 'Recall']].mean().reset_index()

for _, linha in medias_por_preprocessador.iterrows():
    print(f"Preprocessador: {linha['preprocessador']} | Média Accuracy: {linha['mean_test_score']:.4f} | Média Recall: {linha['Recall']:.4f}")


print(f"\n =======  Media total ======= \n Accuracy: {media_acc_total:.4f} | Recall: {media_recall_total:.4f}")

melhor_linhaSVM = df_detalhadoSVM.loc[df_detalhadoSVM['Recall'].idxmax()]

print("\n======= Melhor resultado Recall =======")
print(f"Preprocessador: {melhor_linhaSVM['preprocessador']}")
print(f"Modelo: {melhor_linhaSVM['modelo']}")
print(f"Kernel: {melhor_linhaSVM['kernel']}")
print(f"C: {melhor_linhaSVM['C']}")
print(f"Accuracy (mean_test_score): {melhor_linhaSVM['mean_test_score']:.4f}")
print(f"Recall-score: {melhor_linhaSVM['Recall']:.4f}")
print(f"Tempo médio de treino: {melhor_linhaSVM['mean_fit_time']:.4f} s")
print(f"Tempo médio de score: {melhor_linhaSVM['mean_score_time']:.4f} s")

df_detalhadoSVM.to_csv('df_resultadosSVM.csv', index=False, sep=',')



=== Preprocessador: preprocessador 1 | Kernel: linear ===
modelo   preprocessador kernel   C  mean_test_score   Recall  mean_fit_time  mean_score_time
   SVM preprocessador 1 linear 1.5         0.598959 0.534800     205.640691         7.019500
   SVM preprocessador 1 linear 1.0         0.597831 0.532933     187.211527         7.441445
   SVM preprocessador 1 linear 0.5         0.595283 0.528267     174.513356         7.605440

>>> Medias para o Preprocessador: preprocessador 1 | Kernel: linear <<<
Média Accuracy: 0.5974 | Média Recall: 0.5320

=== Preprocessador: preprocessador 1 | Kernel: poly ===
modelo   preprocessador kernel   C  mean_test_score   Recall  mean_fit_time  mean_score_time
   SVM preprocessador 1   poly 1.5         0.571818 0.616133     153.828377         6.345562
   SVM preprocessador 1   poly 1.0         0.569858 0.605867     173.442150         8.560989
   SVM preprocessador 1   poly 0.5         0.562779 0.578533     171.469141         9.135525

>>> Medias para o Pr

In [8]:
#---------------------------------MLP
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

amostra = x.copy()
amostra['target'] = y
amostra = amostra.sample(50000)

x_sample = amostra.drop('target', axis=1)
y_sample = amostra['target']

modelo_parametrosMLP = {
    'MLP': {
        'modelo': MLPClassifier(),
        'parametros': {
            'activation': ['logistic', 'tanh', 'relu'],
            'max_iter': [2000],
            'hidden_layer_sizes': [(64,64)],
            'random_state': [0],
            'tol': [0.00001],
            'learning_rate_init': [0.001],
            'verbose': [True],
            'alpha': [0.0005],

        }
    }
}

detalhes_MLP = []

for nome_prep, preprocessador in preprocessadores.items():
    x_sample_prep = x_sample.copy()
    x_sample_prep = preprocessador.fit_transform(x_sample_prep)
    x_treinamento_sample, x_teste_sample, y_treinamento_sample, y_teste_sample = train_test_split(x_sample_prep, y_sample, test_size=0.15, random_state=0)

    if hasattr(x_treinamento_sample, "toarray"):
        x_treinamento_sample = x_treinamento_sample.toarray()

    x_treinamento_sample = pd.DataFrame(
        x_treinamento_sample,
        index=y_treinamento_sample.index
    )

    train_df = pd.concat([x_treinamento_sample, y_treinamento_sample], axis=1)

    classe_0 = train_df[train_df['target'] == 0]
    classe_1 = train_df[train_df['target'] == 1]

    fator = round(len(classe_0) / len(classe_1))

    classe_1_bal = pd.concat([classe_1] * fator, ignore_index=True)

    balanced_train_df = pd.concat([classe_0, classe_1_bal], ignore_index=True)
    x_bal_train = balanced_train_df.drop('target', axis=1)
    y_bal_train = balanced_train_df['target']

    for nome_modelo, objeto in modelo_parametrosMLP.items():

        pipeline = Pipeline([
            ('modelo', objeto['modelo']),
        ])

        grid = GridSearchCV(objeto['modelo'], objeto['parametros'], cv=5, scoring='recall_macro', n_jobs=-1)
        grid.fit(x_bal_train, y_bal_train)

        df_MLP= pd.DataFrame(grid.cv_results_)
        y_pred = grid.best_estimator_.predict(x_teste_sample)

        df_MLP['modelo'] = nome_modelo
        df_MLP['preprocessador'] = nome_prep
        df_MLP['activation'] = df_MLP['params'].apply(lambda x: x['activation'])
        
        recall_scores = []
        for params in df_MLP['params']:
            modelo = MLPClassifier(**params)
            modelo.fit(x_bal_train, y_bal_train)
            y_pred = modelo.predict(x_teste_sample)
            recall = recall_score(y_teste_sample, y_pred, average='macro')
            recall_scores.append(recall)

        df_MLP['Recall'] = recall_scores

        detalhes_MLP.append(df_MLP[[
            'modelo', 'preprocessador', 'activation',
            'mean_test_score', 'Recall', 'mean_fit_time', 'mean_score_time',
        ]])

        

df_detalhadoMLP= pd.concat(detalhes_MLP, ignore_index=True)
df_detalhadoMLP = df_detalhadoMLP.sort_values(by=['preprocessador', 'activation'], ascending=[True, True])

for activation, df_sub in df_detalhadoMLP.groupby('activation'):
    print(f"\nActivation: {activation}")
    print(df_sub.to_string(index=False))

    for preproc, df_sub_prep in df_sub.groupby('preprocessador'):
        print(f"\n--- Resultados para Preprocessador: {preproc} ---")
        print(df_sub_prep.to_string(index=False))

    media_acc = df_sub['mean_test_score'].mean()
    media_recall = df_sub['Recall'].mean()
    print(f"Média Accuracy: {media_acc:.4f} | Média Recall: {media_recall:.4f}")

print("\n======= Media por pre-processador =======")
medias_por_preprocessador = df_detalhadoMLP.groupby('preprocessador')[['mean_test_score', 'Recall']].mean().reset_index()

for _, linha in medias_por_preprocessador.iterrows():
    print(f"Preprocessador: {linha['preprocessador']} | Média Accuracy: {linha['mean_test_score']:.4f} | Média Recall: {linha['Recall']:.4f}")


media_recall_total = df_detalhadoMLP['Recall'].mean()
media_acc_total = df_detalhadoMLP['mean_test_score'].mean()
print(f"\n =======  Media total ======= \n Accuracy: {media_acc_total:.4f} | Recall: {media_recall_total:.4f}")

melhor_linhaMLP = df_detalhadoMLP.loc[df_detalhadoMLP['Recall'].idxmax()]

print("\n======= Melhor Resultado Recall =======")
print(f"Preprocessador: {melhor_linhaMLP['preprocessador']}")
print(f"Modelo: {melhor_linhaMLP['modelo']}")
print(f"activation: {melhor_linhaMLP['activation']}")
print(f"Accuracy (mean_test_score): {melhor_linhaMLP['mean_test_score']:.4f}")
print(f"Recall-score: {melhor_linhaMLP['Recall']:.4f}")
print(f"Tempo médio de treino: {melhor_linhaMLP['mean_fit_time']:.4f} s")
print(f"Tempo médio de score: {melhor_linhaMLP['mean_score_time']:.4f} s")


df_detalhadoMLP.to_csv('df_resultadosMLP.csv', index=False, sep=',')


Iteration 1, loss = 0.68042127
Iteration 2, loss = 0.66569182
Iteration 3, loss = 0.66211988
Iteration 4, loss = 0.66062704
Iteration 5, loss = 0.65819796
Iteration 6, loss = 0.65763216
Iteration 7, loss = 0.65783524
Iteration 8, loss = 0.65639479
Iteration 9, loss = 0.65532064
Iteration 10, loss = 0.65456339
Iteration 11, loss = 0.65507152
Iteration 12, loss = 0.65360571
Iteration 13, loss = 0.65333565
Iteration 14, loss = 0.65255790
Iteration 15, loss = 0.65242005
Iteration 16, loss = 0.65179182
Iteration 17, loss = 0.65097283
Iteration 18, loss = 0.65068764
Iteration 19, loss = 0.64973398
Iteration 20, loss = 0.64976171
Iteration 21, loss = 0.64998992
Iteration 22, loss = 0.64948257
Iteration 23, loss = 0.64894991
Iteration 24, loss = 0.64805508
Iteration 25, loss = 0.64814760
Iteration 26, loss = 0.64735861
Iteration 27, loss = 0.64619569
Iteration 28, loss = 0.64751129
Iteration 29, loss = 0.64591002
Iteration 30, loss = 0.64508129
Iteration 31, loss = 0.64550636
Iteration 32, los

C:\Users\Gabriel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.66285843
Iteration 2, loss = 0.65615823
Iteration 3, loss = 0.65390034
Iteration 4, loss = 0.65175088
Iteration 5, loss = 0.65031050
Iteration 6, loss = 0.64879121
Iteration 7, loss = 0.64685075
Iteration 8, loss = 0.64494455
Iteration 9, loss = 0.64261051
Iteration 10, loss = 0.64085118
Iteration 11, loss = 0.63823820
Iteration 12, loss = 0.63602363
Iteration 13, loss = 0.63366986
Iteration 14, loss = 0.63185062
Iteration 15, loss = 0.62933951
Iteration 16, loss = 0.62640821
Iteration 17, loss = 0.62398209
Iteration 18, loss = 0.62097473
Iteration 19, loss = 0.61764012
Iteration 20, loss = 0.61414679
Iteration 21, loss = 0.61125227
Iteration 22, loss = 0.60731496
Iteration 23, loss = 0.60388211
Iteration 24, loss = 0.60053707
Iteration 25, loss = 0.59654378
Iteration 26, loss = 0.59280581
Iteration 27, loss = 0.58915254
Iteration 28, loss = 0.58524247
Iteration 29, loss = 0.58188998
Iteration 30, loss = 0.57838711
Iteration 31, loss = 0.57451524
Iteration 32, los